# Item 01. 가변성을 제한해라

## (1) val 은 Immutable이 아니라 ReadOnly

In [0]:
val readOnlyList = mutableListOf("a", "b", "c")

readOnlyList.add("d")
readOnlyList.add("e")

println(readOnlyList)

In [0]:
val readOnlyListWithDeclareInterfaceList : List<String> = mutableListOf("a", "b", "c")

//readOnlyListWithDeclareInterfaceList.add 인터페이스가 없음

## (2) getter는 smart casting 불가

In [0]:
val firstName : String? = "kim"

val lastName : String? = "amuga"

val fullNameWithDeligate: String?
    get() = firstName?.plus(lastName)

val fullName: String? = firstName?.plus(lastName)

if(fullNameWithDeligate != null) {
    /**
    *  컴파일 에러
    *
    *  getter 로 직접 정의한경우 프로퍼티를 호출할때마다 해당 get func을 실행한다는 의미이며
    *  val은 기본적으로 readOnly이지 immutable이 아니므로 컴파일러가 스마트 캐스팅을 해줄 수 없음!
    *
*/
//    println(fullNameWithDeligate.length)
}

if(fullName != null) {
    println(fullName.length)
}


## (3) 코틀린의 immutable collection 이라 생각한 인터페이스는 사실 readOnly collection일 뿐이다

코틀린은 기본적으로 (ReadOnly) Collection 과 MutableCollection 으로 나뉘며, MutableCollection은 각 ReadOnly Collection들을 구현하며

java에서 구현된 다양한 리스트를 인터페이스로 받게된다.

![상속이미지](../src/main/resources/abstractList.png)

### 자바의 java.util.ArrayList vs java.util.Arrays.ArrayList

- java.util.ArrayList는 동적할당 리스트, java.util.List 인터페이스를 구현
- java.util.Arrays.ArrayList는 사이즈 정적 할당 리스트, java.util.AbstractList 클래스를 상속하며, java.util.List 인터페이스를 구현


In [39]:
import java.util.*

val readOnlyList = listOf(3,4,5)
// == java.util.Arrays.asList(3,4,5) - listOf는 정적 사이즈 할당 리스트 구현체이며, 인터페이스는 코틀린 readOnly인 List로 간주된다

println(readOnlyList[0])
//readOnlyList[1] = 1 readOnly이므로 해당 타입에서는 당연히 참조 재할당 불가능


/**
*
* readOnlyList.add는 인터페이스가 없어서 변경 불가능
*
* 하지만, 실 구현체자체는 java.util.Arrays.ArrayList 이므로
*
* 다운캐스팅이 가능해진다
*/


if(readOnlyList is MutableList) { // 다운 캐스팅시 if문을 통과!
    readOnlyList[1] = 10 // java.util.Arrays.ArrayList이므로 수정도 된다!
}

println(readOnlyList)

if(readOnlyList is MutableList) { // 다운 캐스팅시 if문을 통과!
//    readOnlyList.add(11) // 이건 정적할당 List이므로 사이즈가 변경 불가능하여 추가 제거가 안되므로 런타임시 AbstractList 에러,
}


3
[3, 10, 5]



결론적으로 코틀린의 List, Map 등의 인터페이스는 immutable을 완전히 보장한다고 할 수 없다.

다만 `val readOnlyList : List = listOf(3,4,5)` 와 같이 인터페이스를 선언하게되면 다운캐스팅시 컴파일 에러를 발생시키므로 우회적으로 방어는 가능하다.
- kotlin 1.8 기준

`
val readOnlyDeclaredList : List<Int> = listOf(1,2,3,4) // jupiter에서는 안되지만 코틀린 1.8기준으론 캐스팅 가능

if(readOnlyDeclaredList is MutableList) { // 다운 캐스팅시 if문을 통과!
    readOnlyDeclaredList[1] = 10 // java.util.Arrays.ArrayList이므로 수정도 된다!
}

println(readOnlyDeclaredList)
`
### 실무적으론?

readOnly -> mutable 콜렉션 다운캐스팅은 기본적으로 코틀린이 제공하는 api를 거스르는 행위이고, 버전에 따라 앞으로 어떻게 행위가 바뀔지 모르기때문에 절대 위와같이 코드를 작성하지 말고,

만약 readOnly콜렉션을 변경해야할 일이 있을경우엔

`toMutableList()` 와 같이 copy를 통해 변경하도록하자.

이때 주의할점은 해당행위는 deepCopy라고 할수 없으므로 조심해야한다.

### 진짜 불변 콜렉션이 필요하다면?

- Guava:
구글이 개발한 Guava 라이브러리에서 불변 컬렉션을 제공 ImmutableList, ImmutableSet, ImmutableMap 등

- Arrow:
코틀린용 함수형 프로그래밍 라이브러리 Arrow도 불변 컬렉션을 제공:  NonEmptyList, SetK, MapK 등




## (4) immutable 객체와 동등성

기본적으로 인스턴스는 변경될때 해시코드도 변경되기때문에, 해시값을 가지고 동등성를 판단하는 로직이 있다면 문제가 될 수 있다.

따라서 해시코드 재정의를 통해 동등성을 관리해주는게 바람직한데, 이 작업은 매우 귀찮으므로 val 프로퍼티를 가진 immutable data class를 만들고

변경시엔 이 data class를 copy를 통해 변경하는 방식으로 구현하는것이 코틀린이 권장하는 인스턴스 변경방법이다.


## (5) 다른 종류의 변경가능지점


In [0]:

val valMutableList = mutableListOf<Int>()
var varlist = listOf<Int>()

valMutableList.add(1)
varlist = (varlist + 1)

println(valMutableList)
println(varlist)

전자는 mutableList이며

후자는 ReadOnlyList를 참조하는 프로퍼티의 값을 변경할수 있다.

변경지점의 깊이가 다르다고 볼 수 있는데,두 코드 모두 동시성 환경에서 안전한코드가 아니다.

책에서는 Collection 구현체에대한 threadSafe 보장을 확신할 수 없으므로, 후자쪽으로 코드를 작성하고

직접 동시성 제어에대한 코드를 작성하는게 바람직하다고 주장하는데,

일반적으로 동시성 제어를 하는 코드를 넣거나, treadSafe한 자료구조를 쓰면 되는거 아닌가?

후자쪽은 새로운 Collection을 만드는 오버헤드도 고려해야해서 동의가 안된다.



## (6) 애초에 변경 가능 지점을 노출하지 않기

- 방어적 복사본을 제공
- Collection이라면 업캐스팅을 통해 ReadOnly 타입으로 만들기